<a href="https://colab.research.google.com/github/Vivek084c/image_processing/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
import shutil

In [2]:
# Global parameters
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir("data/input")
create_dir("data/catogories")


In [3]:
for i in range(300):
  shutil.copy(f"/content/drive/MyDrive/dataset/vi/input/img_{i}.jpg", "/content/data/input")
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
for i in range(300):
  shutil.copy(f"/content/drive/MyDrive/dataset/vi/categories/img_{i}.png", "/content/data/catogories")
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:

print(len(glob(os.path.join("/content", "data", "input", "*"))))
print(len(glob(os.path.join("/content", "data","catogories", "*"))))

300
300


In [6]:


def load_dataset(path, split=0.2):
    train_x = sorted(glob(os.path.join(path, "input", "*")))[:300]
    train_y = sorted(glob(os.path.join(path, "catogories", "*")))[:300]

    split_size = int(split * len(train_x))

    train_x, valid_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))  # Resize to (IMG_W, IMG_H)
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))  # Resize to (IMG_W, IMG_H)

    # Processing the mask
    output = []
    for color in COLORMAP:
        cmap = np.all(np.equal(x, color), axis=-1)
        output.append(cmap)
    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)
    return output

def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_mask(y)

        return x, y

    image, mask = tf.numpy_function(f, [x, y], [np.float32, np.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=2000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [7]:





def fcn8(input_shape, num_classes=18):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    pool1 = x

    # Block 2
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    pool2 = x

    # Block 3
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    pool3 = x

    # Block 4
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    pool4 = x

    # Block 5
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    pool5 = x

    # Convolutional layers to replace FC layers
    x = layers.Conv2D(4096, (7, 7), activation='relu', padding='same')(pool5)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(4096, (1, 1), activation='relu', padding='same')(x)
    x = layers.Dropout(0.5)(x)

    # Classifying layer
    x = layers.Conv2D(num_classes, (1, 1), kernel_initializer='he_normal')(x)
    score_fr = x

    # Deconvolution layers
    x = layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(score_fr)
    score_pool4 = layers.Conv2D(num_classes, (1, 1), kernel_initializer='he_normal')(pool4)
    x = layers.Add()([x, score_pool4])

    x = layers.Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    score_pool3 = layers.Conv2D(num_classes, (1, 1), kernel_initializer='he_normal')(pool3)
    x = layers.Add()([x, score_pool3])

    # Final upsampling
    x = layers.Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(8, 8), padding='same')(x)

    # Output layer
    outputs = layers.Activation('softmax')(x)

    model = models.Model(inputs, outputs)

    return model

In [ ]:
if __name__ == "__main__":
    # Seeding the environment
    np.random.seed(41)
    tf.random.set_seed(41)

    # Directory to save files
    create_dir("files")

    # Hyperparameters
    IMG_H = 640
    IMG_W = 832
    NUM_CLASSES = 18
    input_shape = (IMG_H, IMG_W, 3)
    batch_size = 8
    lr = 1e-4
    num_epoch = 30

    dataset_path = "/content/data"
    model_path = os.path.join("files", "model.keras")
    csv_path = os.path.join("files", "data.csv")

    # Loading the dataset
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    # Colormap processing
    COLORMAP = [
        [0, 0, 0], [0, 0, 128], [0, 0, 255], [0, 85, 0], [51, 0, 170],
        [0, 85, 255], [85, 0, 0], [221, 119, 0], [0, 85, 85], [85, 85, 0],
        [0, 51, 85], [128, 86, 52], [0, 28, 0], [255, 0, 0], [221, 170, 51],
        [225, 225, 0], [170, 255, 85], [85, 255, 170]
    ]
    CLASSES = [
        "Background", "Hat", "Hair", "Sunglasses", "Upper-clothes", "Skirt",
        "Pants", "Dress", "Belt", "Left-shoe", "Right-shoe", "Face", "Left-leg",
        "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"
    ]

    # Dataset pipeline
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    # Model
    model = fcn8(input_shape=input_shape, num_classes=NUM_CLASSES)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr)
    )
    model.summary()

    # Training
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=num_epoch,
        callbacks=callbacks
    )

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 640, 832, 3)]        0         []                            
                                                                                                  
 conv2d_54 (Conv2D)          (None, 640, 832, 64)         1792      ['input_4[0][0]']             
                                                                                                  
 conv2d_55 (Conv2D)          (None, 640, 832, 64)         36928     ['conv2d_54[0][0]']           
                                                                                                  
 max_pooling2d_15 (MaxPooli  (None, 320, 416, 64)         0         ['conv2d_55[0][0]']           
 ng2D)                                                                                      